In [1]:
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPool2D
from keras.utils import to_categorical

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

### Data preprocess

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

px.imshow(np.hstack(X_train[:10])).update_coloraxes(showscale=False).show()

w, h = X_train[0].shape
print(f"Single image shape is {w} x {h}")

Single image shape is 28 x 28


Convert target to binary array

In [3]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

Normalize features

In [4]:
max_val = np.max(X_train)
X_train_scaled = X_train / max_val
X_test_scaled = X_test / max_val

### Sequential with dense layers only (MLP)

In [5]:
model = Sequential([
    Input((w, h, 1)),
    Flatten(),
    Dense(128, activation='sigmoid'),
    Dense(128, activation='sigmoid'),
    Dense(10, activation='softmax'),
])

model.summary()

2025-02-06 00:34:17.238056: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-02-06 00:34:17.238078: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-02-06 00:34:17.238082: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-02-06 00:34:17.238097: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-06 00:34:17.238108: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,282 (462.04 KB)

 Trainable params: 118,282 (462.04 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
with tf.device("/GPU:0"):
    history = model.fit(X_train_scaled, y_train_cat, epochs=10, batch_size=256, validation_data=(X_test_scaled, y_test_cat))

Epoch 1/10


2025-02-06 00:34:17.689582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.5562 - loss: 1.4622 - val_accuracy: 0.7941 - val_loss: 0.5929
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8159 - loss: 0.5280 - val_accuracy: 0.8262 - val_loss: 0.4802
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8478 - loss: 0.4264 - val_accuracy: 0.8434 - val_loss: 0.4371
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8606 - loss: 0.3893 - val_accuracy: 0.8523 - val_loss: 0.4106
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8670 - loss: 0.3711 - val_accuracy: 0.8561 - val_loss: 0.3983
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8723 - loss: 0.3574 - val_accuracy: 0.8570 - val_loss: 0.3940
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8739 - loss: 0.3475 - val_accuracy: 0.8639 - val_loss: 0.3752
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8816 - loss: 0.3281 - val_accuracy: 0.865

In [8]:
accuracy_history = pd.DataFrame({
    'train': history.history['accuracy'],
    'test': history.history['val_accuracy']
})

loss_history = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(accuracy_history, title='Accuracy').show()
px.line(loss_history, title='Loss').show()

### Sequential with conv and pooling layers 

In [9]:
model = Sequential([
    Input((w, h, 1)),
    Conv2D(filters=16, kernel_size=3, strides=1, padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=32, kernel_size=3, strides=1, padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=3, strides=1, padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(64, activation='elu'),
    Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,874 (237.79 KB)

 Trainable params: 60,874 (237.79 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
history = model.fit(X_train_scaled, y_train_cat, epochs=15, batch_size=256, validation_data=(X_test_scaled, y_test_cat))

Epoch 1/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.6633 - loss: 0.9879 - val_accuracy: 0.8318 - val_loss: 0.4651
Epoch 2/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.8448 - loss: 0.4315 - val_accuracy: 0.8521 - val_loss: 0.4225
Epoch 3/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.8728 - loss: 0.3611 - val_accuracy: 0.8720 - val_loss: 0.3603
Epoch 4/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.8857 - loss: 0.3215 - val_accuracy: 0.8771 - val_loss: 0.3361
Epoch 5/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.8949 - loss: 0.2921 - val_accuracy: 0.8830 - val_loss: 0.3183
Epoch 6/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.8999 - loss: 0.2782 - val_accuracy: 0.8880 - val_loss: 0.3100
Epoch 7/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9077 - loss: 0.2562 - val_accuracy: 0.8895 - val_loss: 0.3070
Epoch 8/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9089 - loss: 0.2496 - val_accu

In [12]:
accuracy_history = pd.DataFrame({
    'train': history.history['accuracy'],
    'test': history.history['val_accuracy']
})

loss_history = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(accuracy_history, title='Accuracy').show()
px.line(loss_history, title='Loss').show()

### LeNet (for ditits MNIST)

<img src='images/LeNet.png' width=500px>

In [13]:
model = Sequential([
    Input((w, h, 1)),
    Conv2D(filters=6, kernel_size=5, strides=1, padding='valid'),
    MaxPool2D(),
    Conv2D(filters=16, kernel_size=5, strides=1, padding='valid'),
    MaxPool2D(),
    Flatten(),
    Dense(120, activation='sigmoid'),
    Dense(84, activation='sigmoid'),
    Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 16)       │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 120)            │        30,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,426 (173.54 KB)

 Trainable params: 44,426 (173.54 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

w, h = X_train[0].shape

max_val = X_train.max()
X_train_scaled = X_train / max_val
X_test_scaled = X_test / max_val

px.imshow(np.hstack(X_train_scaled[:10])).update_coloraxes(showscale=False).show()

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
history = model.fit(X_train_scaled, y_train_cat, epochs=10, validation_data=(X_test_scaled, y_test_cat))

In [ ]:
accuracy_history = pd.DataFrame({
    'train': history.history['accuracy'],
    'test': history.history['val_accuracy']
})

loss_history = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(accuracy_history, title='Accuracy').show()
px.line(loss_history, title='Loss').show()